## [1] kaggle 사용을 위한 setting

In [0]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=cbf095646836e3e55357d0c5b6835b9a6d53bd0d6af50ec8bb642663f7d9e41f
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [0]:
!mkdir -p ~/.kaggle                   #캐글 폴더 생성
!cp kaggle.json ~/.kaggle/            # 폴더에 kaggle.json을 복붙하기
! chmod 600 ~/.kaggle/kaggle.json     # 복붙한 것의 권한을 주기
!kaggle -v

Kaggle API 1.5.6


In [0]:
!kaggle competitions download -c logistic-classification-diabetes

!unzip logistic-classification-diabetes.zip   #폴더이름.zip

  0% 0.00/16.3k [00:00<?, ?B/s]
100% 16.3k/16.3k [00:00<00:00, 14.4MB/s]
Archive:  logistic-classification-diabetes.zip
  inflating: submission_form.csv     
  inflating: test_data.csv           
  inflating: train.csv               


## [2] 데이터 로더

In [0]:
import numpy as np
import torch
import torch.optim as optim
import pandas as pd

xy = np.loadtxt('train.csv', delimiter=",", dtype=np.float32 ,skiprows=1, usecols=range(1,10))
print(xy)


x_data=xy[:,0:-1]
y_data=xy[:,[-1]].squeeze()         # pandas_xy.loc[:,9]

x_train = torch.FloatTensor(x_data)  
y_train = torch.LongTensor(y_data)  

[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.176471   0.879397  -0.180328  ... -0.36123   -0.566667   0.       ]
 [-0.647059   0.738693   0.278689  ... -0.238258  -0.666667   0.       ]
 [ 0.176471  -0.0552764  0.180328  ... -0.558497   0.166667   1.       ]]


In [0]:
print(x_train[:5])   
print(y_train[:5])

print(x_train.shape)
print(y_train.shape)


tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
tensor([0, 1, 0, 1, 0])
torch.Size([709, 8])
torch.Size([709])


## [3] 모델 학습

In [0]:
import torch.nn.functional as F  # for softmax

#feature 8개, 2개의 클래스(0,1)

nb_class = 2
nb_data = len(y_train)

W = torch.zeros((8, nb_class), requires_grad=True)  
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)   #lr=0.01

nb_epochs = 10000               
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    
    # cost 표현번 1번 예시
      # one-hot encoding.
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()


    # cost로 H(x) 개선 (w 업데이트)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))



Epoch    0/10000 Cost: 0.693147
Epoch 1000/10000 Cost: 0.614406
Epoch 2000/10000 Cost: 0.597775
Epoch 3000/10000 Cost: 0.586016
Epoch 4000/10000 Cost: 0.576891
Epoch 5000/10000 Cost: 0.569821
Epoch 6000/10000 Cost: 0.564295
Epoch 7000/10000 Cost: 0.559904
Epoch 8000/10000 Cost: 0.556352
Epoch 9000/10000 Cost: 0.553432
Epoch 10000/10000 Cost: 0.550995


## [4] 정답예측

In [0]:
# 학습된 W,b를 통한 클래스 예측

hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1) 
          # torch.argmax(): hypothesis(=wx)에서 가장 큰 값의 인덱스를 return

#print(hypothesis) # 범주형데이터로 바뀌기 전. wx를 구한 상태.
#print(predict)    # 예측한 클래스 
#print(y_train)    # 실제 데이터.

# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction[:5])

accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

# lr=0.01, 에폭=1000, 정확도는 76.45%
# lr=0.01, 에폭 10000, 정확도는 77.15%

tensor([ True,  True,  True,  True,  True,  True, False,  True,  True, False,
         True,  True, False,  True, False,  True, False,  True, False,  True,
         True,  True, False, False,  True,  True,  True,  True,  True, False,
         True,  True,  True,  True, False, False, False, False,  True, False,
         True,  True, False,  True,  True,  True, False,  True,  True,  True,
         True,  True, False,  True,  True,  True, False,  True,  True, False,
         True,  True, False,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False, False,  True,
         True,  True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True, False, False,  True,  True, False, False, 

## 데이터에 대한 TEST
캐글의 TEST 셋을 가지고 예측값을 구하기

In [0]:
# usecol=range(1,9)  10이 아님.
test=np.loadtxt('test_data.csv',delimiter=",", dtype=np.float32, skiprows=1, usecols=range(1,9))

x_data=test     # test_data.csv 에는 라벨이 없기때문에 라벨에 관한것을 하면 오류가 뜰 것임

x_data=np.array(x_data)
x_test=torch.FloatTensor(x_data)

hypothesis=F.softmax(x_test.matmul(W) +b, dim=1)
predict=torch.argmax(hypothesis, dim=1)
predict    

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1], grad_fn=<NotImplemented>)

## [5] 캐글에 제출하기


- test 데이터 셋의 포맷에 맞추기.
submit_sample.csv


In [0]:
import pandas as pd

submit=pd.read_csv('submission_form.csv')
submit               # id와 expected의 헤더를 가지고 있다.

,ID,Label
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN
5,5,NaN
6,6,NaN
7,7,NaN
8,8,NaN
9,9,NaN


In [0]:
predict= predict.detach().numpy().reshape(-1,1)

id=np.array([i for i in range(50)]).reshape(-1,1)

result=np.hstack([id,predict])
df=pd.DataFrame(result, columns=["id","Label"])
df.to_csv('result.csv', index=False, header=True)

In [0]:
result

array([[ 0,  1],
       [ 1,  1],
       [ 2,  1],
       [ 3,  1],
       [ 4,  1],
       [ 5,  1],
       [ 6,  1],
       [ 7,  1],
       [ 8,  1],
       [ 9,  1],
       [10,  1],
       [11,  1],
       [12,  1],
       [13,  1],
       [14,  0],
       [15,  1],
       [16,  1],
       [17,  1],
       [18,  1],
       [19,  1],
       [20,  1],
       [21,  1],
       [22,  0],
       [23,  1],
       [24,  1],
       [25,  0],
       [26,  0],
       [27,  1],
       [28,  0],
       [29,  1],
       [30,  0],
       [31,  0],
       [32,  0],
       [33,  1],
       [34,  1],
       [35,  0],
       [36,  0],
       [37,  1],
       [38,  1],
       [39,  1],
       [40,  1],
       [41,  0],
       [42,  1],
       [43,  0],
       [44,  1],
       [45,  1],
       [46,  1],
       [47,  1],
       [48,  1],
       [49,  1]])

In [0]:
! kaggle competitions submit -c logistic-classification-diabetes -f result.csv -m 'first 16010064 이주희'

100% 249/249 [00:07<00:00, 32.3B/s]
Successfully submitted to logistic classification : diabetes

In [0]:
# kaggle의 leader보드에서 확인하기